In [1]:
import pickle
import sys
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

sys.path.append(os.path.abspath("../tokenizer/"))
from tok import RegexTokenizer


with open('../tokenizer/toks_vocab_3k.pkl', 'rb') as f: #loading vocab from tokenizer
    tok_dict = pickle.load(f)


with open('../poems2.txt', 'r', encoding='utf-8') as f:
    poems = f.read()

In [2]:
#creating tokenizer and adding vocab and merges trained a priori
vocab3k = dict(list(tok_dict['vocab'].items()))#[:3000]) #cutting the vocab size to 3k
merges3k= dict(list(tok_dict['merges'].items()))#[:(3000-256)])

tokenizer = RegexTokenizer()
tokenizer.vocab = vocab3k
tokenizer.merges = merges3k

#encoding the dataset
encoded_poems = tokenizer.encode(poems)

In [3]:
len(tokenizer.special) 
len(tokenizer.vocab)

3000

In [4]:
#defining hyperparameters0

parameters = {'block_size':8, 'n_embd': 32, 'n_head':4, 'n_layer':4, 'dropout':0.2, 'vocab_size': len(tokenizer.vocab)+1}#+1 adds end token
batch_size = 32
block_size = parameters['block_size']
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [5]:
#making it a torch tensor and splitting into training and validation sets
data = torch.tensor(encoded_poems, dtype=torch.long)
print(f'Dataset with {len(poems)} now is represented by {len(data)} tokens')
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Dataset with 1355622 now is represented by 425406 tokens


In [6]:
#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [7]:
from transformer_special import GPTLanguageModel

model = GPTLanguageModel(parameters)
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [8]:
#training
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 8.1793, val loss 8.1917
step 500: train loss 6.6707, val loss 6.7539
step 1000: train loss 6.2464, val loss 6.3503
step 1500: train loss 5.9544, val loss 6.0734
step 2000: train loss 5.7338, val loss 5.8926
step 2500: train loss 5.5954, val loss 5.7569
step 3000: train loss 5.4718, val loss 5.6300
step 3500: train loss 5.3717, val loss 5.5792
step 4000: train loss 5.2978, val loss 5.4946
step 4500: train loss 5.2419, val loss 5.4414


In [10]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(tokenizer.decode(m.generate(context, max_new_tokens=500)[0].tolist()))

  pradar bem inda de tudo a mim?
Erio
É conrias infeliz, Di,
E Cegis a, porque morrer, talvez ainda o com todo trás,
Och pensar já como nem colloça
Como essem do sos dos viver, me seempados!
Se além pelo mistério sentia indo —
Um colmas é consciência da momento,
Porque obvelçãoatista.

Para vida. mistério ouvitros da alma é uma gente alta,
Nemxí nada.
Nasigfa suprdia no dia contra os jo ou lento fé,
Granões...

Que há
Mhe quando hoje me homens tido,
Dentes nuvens e do monte.
Não: como nem queramente só humana isto de seus seus me lELenas querpiri quase: são

E sentir, enfim então o ter doente que sorr?
Só a me vla de guerra velha e-labil eu o que eu,
Nopressão na loucura seufrenda
E exasção — morte riturgido, Deus,
Sicia evenso que a corslon?
<end>Fernando Pessoa
Aoismo antes de Camposialidade, humana,
Como ao que pelas mim, 
E nesta dever, não vê, homem e as chamar deca àetas! Eh-eh-eh-eh-eh-eh-eh-eh-eh which ar seja
 Whuresse o monela
Com mãos, volimo a sua florri this
E
O desubos qu

In [11]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
token_embedding_table.weight 	 torch.Size([3001, 32])
position_embedding_table.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.0.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.0.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.0.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.0.value.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.1.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.1.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.1.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.1.value.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.2.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.2.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.2.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.2.value.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.3.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.3.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.3.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.3.value.weight 	 torch.Size([8, 32])
blocks.0.sa.proj.weight 	 torch.Size

In [12]:
torch.save(model, 'small_model_special.pt')